In [1]:
import pandas as pd
import datetime#, timedelta

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

In [2]:
housing_Crime_Data = pd.read_csv(r"C:\Users\tochi\Desktop\Crime_Housing.csv")
housing_Crime_Data.describe()

,price_paid
count,1.344350e+05
mean,9.119420e+05
std,5.360636e+06
min,1.000000e+02
25%,3.170000e+05
50%,4.350000e+05
75%,6.450000e+05
max,2.550000e+08


In [3]:
housing_Crime_Data.dtypes

Month            object
Crime.type       object
postcode         object
price_paid        int64
property_type    object
new_build        object
town             object
district         object
dtype: object

In [4]:
housing_Crime_Data.columns

Index(['Month', 'Crime.type', 'postcode', 'price_paid', 'property_type',
       'new_build', 'town', 'district'],
      dtype='object')

In [5]:
housing_Crime_Data['Crime.type'] = pd.Categorical(housing_Crime_Data['Crime.type'] )
housing_Crime_Data['postcode'] = pd.Categorical(housing_Crime_Data['postcode'] )
housing_Crime_Data['Month'] = pd.Categorical(housing_Crime_Data['Month'])
housing_Crime_Data['property_type'] = pd.Categorical(housing_Crime_Data['property_type'] )
housing_Crime_Data['new_build'] = pd.Categorical(housing_Crime_Data['new_build'] )
housing_Crime_Data['district'] = pd.Categorical(housing_Crime_Data['district'] )
housing_Crime_Data['town'] =  pd.Categorical(housing_Crime_Data.loc[:,'town'])


In [6]:
housing_Crime_Data.dtypes

Month            category
Crime.type       category
postcode         category
price_paid          int64
property_type    category
new_build        category
town             category
district         category
dtype: object

In [8]:
Housing_CrimeData_Clean_Cat = housing_Crime_Data.select_dtypes(include=['category'])
Housing_CrimeData_Clean_num = housing_Crime_Data.select_dtypes(exclude=['category'])
X_encoded = pd.get_dummies(Housing_CrimeData_Clean_Cat)

In [9]:
frames = [X_encoded, Housing_CrimeData_Clean_num]
combo_enc = pd.concat(frames, axis = 1)

In [10]:
Housing_CrimeData_Clean_Cat_freq = Housing_CrimeData_Clean_Cat.copy()
for c in Housing_CrimeData_Clean_Cat_freq.columns.to_list():
    Housing_CrimeData_Clean_Cat_freq[c] = Housing_CrimeData_Clean_Cat_freq.groupby(c).transform('count')/len(Housing_CrimeData_Clean_Cat_freq[c])


In [11]:
frames_freq = [Housing_CrimeData_Clean_Cat_freq, Housing_CrimeData_Clean_num]
combo_enc_freq = pd.concat(frames_freq, axis = 1)
combo_enc_freq.shape

(134435, 8)

In [12]:
combo_enc_freq.tail()
combo_enc_freq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134435 entries, 0 to 134434
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Month          134435 non-null  float64
 1   Crime.type     134435 non-null  float64
 2   postcode       134435 non-null  float64
 3   property_type  134435 non-null  float64
 4   new_build      134435 non-null  float64
 5   town           134435 non-null  float64
 6   district       134435 non-null  float64
 7   price_paid     134435 non-null  int64  
dtypes: float64(7), int64(1)
memory usage: 8.2 MB


In [13]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(combo_enc_freq.filter(items =[ 'Month', 'Crime.type', 'postcode', 'property_type',
       'new_build', 'town', 'district']), combo_enc_freq['price_paid'], random_state=0)

In [14]:
from sklearn.neural_network import MLPRegressor

In [15]:
from sklearn.datasets import make_regression

In [43]:
regr = MLPRegressor(random_state=1, max_iter=15000).fit(Xtrain, ytrain)

C:\Users\tochi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [44]:
regr.predict(Xtest)

array([ 672271.06936196, 3195774.85376639,  489987.51534799, ...,
       3268704.75471761,  279349.94606583,  555175.15235468])

In [45]:
regr.score(Xtest, ytest)

0.027650184831768065

In [51]:
#R_Square and Adjusted R Square
import statsmodels.api as sm
X_addC = sm.add_constant(Xtest)
result = sm.OLS(ytest, X_addC).fit()
print(result.rsquared, result.rsquared_adj)


0.019493542853790125 0.019289276754566154


In [69]:
from sklearn.metrics import mean_squared_error
import math
pred_y = regr.predict(Xtest)
print(mean_squared_error(ytest, pred_y))
print(math.sqrt(mean_squared_error(ytest, pred_y)))


30351495581054.434
5509219.14440281


In [67]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(ytest, pred_y))

807765.0093318874


In [ ]:
Root Mean Square Error= 5509219.14440281
               MAE metric mean absolute error= 807765.0093318874
    Mean Square Error = 30351495581054.434
            R square is 1.9%